### Michipicoten, Lake Superior 

**Station Name:** 	mchn

**Location:** Michipicoten Harbor, Ontario, Canada

**Archive:**  [SOPAC](http://sopac-csrc.ucsd.edu/index.php/sopac/) 

**Ellipsoidal Coordinates:**

- Latitude: 47.961

- Longitude: -84.901

- Height: 152.019 m

[Station Page at Natural Resources Canada](https://webapp.geod.nrcan.gc.ca/geod/data-donnees/station/report-rapport.php?id=M093001)

[Station Page at Nevada Geodetic Laboratory](http://geodesy.unr.edu/NGLStationPages/stations/MCHN.sta)

[Google Maps Link](https://goo.gl/maps/mU5GbsvMsLfe5buQ7) 

<p align=center>
<img src="../../../data/mchn_monu-cors.png" width="500"/>
</P>

In [ ]:
import json
import os
import sys

import ipywidgets as widgets
import numpy as np
import seaborn as sns;

sns.set_theme(style="whitegrid");
import matplotlib.pyplot as plt
from scipy import stats

# We are including our repository bin to the system path so that we can import the following python modules
bin_path = os.path.abspath(os.path.join('../../../bin'))
if bin_path not in sys.path:
    sys.path.append(bin_path)

import gnssrefl_helpers
from plotmchn import readfiles, addnans, getrms

#Making sure environment variables are set - this is required to run the gnssrefl code
exists = gnssrefl_helpers.check_environment()
if exists == False:
    gnssrefl_helpers.set_environment(refl_code="../../..", orbits="../../../orbits", exe="../../../bin/exe")

# Set local variable of refl_code location
refl_code_loc = os.environ['REFL_CODE']

# import gnssrefl functions
from gnssrefl.rinex2snr_cl import rinex2snr
from gnssrefl.quickLook_cl import quicklook
from gnssrefl.make_json_input import make_json
from gnssrefl.gnssir_cl import gnssir
from gnssrefl.daily_avg_cl import daily_avg
from gnssrefl.installexe_cl import installexe

#@formatter:off
%matplotlib inline

In [ ]:
# import the crx2rnx file which is dependant on your working OS - this is required to run the gnssrefl code
# If in docker environment, then you do not need to download crxnrnx
try:
    os.environ['DOCKER']
except KeyError:
    sys = gnssrefl_helpers.get_sys()
    installexe(sys)

Fast Mode - this will download data weekly instead of daily


In [ ]:
weekly = widgets.Checkbox(value=True, description='Fast Mode', disabled=False, indent=False)
display(weekly)

### Data Summary

Station mchn is operated by [NRCAN](https://www.nrcan.gc.ca/home).
The station overlooks Lake Superior in a favorable location for measuring seasonal water levels.
This site only tracks legacy GPS signals. 

More information on mchn can be obtained 
from the [GNSS-IR Web App](https://gnss-reflections.org/api?example=mchn),
where mchn is one of the test cases. 

For GNSS reflectometry, you need to set an azimuth and elevation angle mask.
The azimuths are chosen to ensure that the reflected signals reflect off the surface of interest.

Here is a good start on an elevation and azimuth angle mask:

In [ ]:
%%html
<iframe src="https://gnss-reflections.org/rzones?station=mchn&msl=on&RH=7&eang=2&azim1=80&azim2=180" width="1000" height="600"></iframe>

### Take a quick look at the data

If you know where the data are stored (i.e. sopac), it is better (faster) to set that flag.
Since the receiver only tracks GPS signals, there is no need to specify gnss orbits.

In [ ]:
# set variables so we can reuse them in other parts of the code
station = 'mchn'
rinex2snr(station, 2019, 205, archive='sopac')

Examine the spectral characteristics of the SNR data for the default settings
[(For details on quickLook output.)](../../docs/quickLook_desc.md):

In [ ]:
values, metrics = quicklook(station, 2019, 205)

If you explored the web app example, this clearly does not look like the results you saw there. These clearly do not look like good results. When we run quicklook, there are several default values that do not work for every station.

You can see the default values by running the following cell:

In [ ]:
quicklook?

Look closely at the station photo and the x-axis of the periodograms, then change the range of reflector heights for **quickLook**:

In [ ]:
vals, metrics = quicklook(station, 2019, 205, h1=2, h2=8)

The water is ~6.5 meters below the antenna. You can see from the top plot that the good retrievals (in blue) 
very clearly show you which azimuths are acceptable and which are not.  The middle plot shows the peak to noise 
ratio, which we would like to at least exceed 3. And here again, the bad retrievals are always below this level.
The amplitudes in the bottom plot indicate that 8 is an acceptable minimal value.


### Analyze the Data

The data from 2013 will be analyzed here as a test case.  Begin by generating the SNR files.
The resulting SNR files are stored in $REFL_CODE/2013/snr/mchn.

In [ ]:
year = 2013
doy = 1
doy_end = 365

rinex2snr(station, year, doy, doy_end=doy_end, archive='sopac', weekly=weekly.value)

Analysis parameters are set up with <code>make_json</code>

In [ ]:
make_json?

In [ ]:
lat = 47.961
long = -84.901
height = 152.019

make_json(station, lat, long, height, h1=3, h2=10, l1=True, peak2noise=3, ampl=8)

While most of the analysis settings can be done by the command 
line, you can see that the azimuths have been set by
hand to be limited to 80-180 degrees. Although it is possible to get good reflections beyond 
180 degrees, the photographs suggest barriers are present in that region. 

In [ ]:
# This is the json file that was created with the defaults/parameters you set above
json_file = f'{refl_code_loc}/input/{station}.json'

# Now lets edit the json file
with open(json_file, "r") as myfile:
    file = json.load(myfile)
    
# Here is where we can 'hand edit' values in the json file
# lets edit the azimuths. We set these values by looking at the metrics qc plot above
file['azval'] = [80,180]
os.remove(json_file)

with open(json_file, 'w') as f:
    json.dump(file, f, indent=4)
    
# now lets view it again and note the difference
with open(json_file, "r") as myfile:
    file = json.load(myfile)

file

Now that the analysis parameters are set, run <code>gnssir</code> to save the reflector height (RH) output for each day in 2013.

In [ ]:
gnssir(station, year, doy, doy_end=doy_end)

The daily output files are stored in $REFL_CODE/2013/results/mchn.
You can optionally see lots of SNR data with the plt=True option.

In [ ]:
#example of setting plt=True
gnssir(station, year, 195, plt=True)

For a lake, it is appropriate to use the daily average. Our utility for computing a daily average requires a value
for the median filter and a minimum number of tracks. If the median value is set to be large (2 meters), you can see 
large outliers: 

In [ ]:
daily_avg(station, 2, 10)

A more reasonable result is obtained with a 0.25-meter median filter and the 12-track requirement. If you want to save 
the daily averages to a specific file, use the txtfile= option. Otherwise it will use a default location (which is printed to the screen)

In [ ]:
daily_avg(station, 0.25, 12, txtfile='mchn-dailyavg.txt')

The number of tracks required will depend on the site. Here the azimuth is restricted because  of the location of the antenna.
Please note that these reflections are from ice in the winter and water during the summer. Surface 
bias corrections (ice, snow) will be implemented in the software in the future. Until then, please take 
this into account when interpreting the results.

There is a [tide gauge](https://tides.gc.ca/eng/Station/Month?sid=10750) at this site. The data can be 
downloaded from [this link](http://www.isdm-gdsi.gc.ca/isdm-gdsi/twl-mne/inventory-inventaire/interval-intervalle-eng.asp?user=isdm-gdsi&region=CA&tst=1&no=10750). 
Please select the daily mean water level, as there are restrictions on hourly data (more information is available on the download page). 
We have downloaded [the 2013 data](../../../data/10750-01-JAN-2013_slev.csv).

The water levels measured by the traditional tide gauge and GNSS-IR are shown here:

In [ ]:
tidedates, waterlevel, mchndates, reflht = readfiles(file1='../../../data/10750-01-JAN-2013_slev.csv', 
                                                                         file2=f'../../../Files/{station}/{station}-dailyavg.txt')
#pad missing days with nan
ymd, padded_rh = addnans(mchndates, reflht)
ymd, padded_wl = addnans(tidedates, waterlevel)    

#create numpy array objects
rh_array = np.array(padded_rh)
wl_array = np.array(padded_wl)    

#get linear regression (use scipy but mask out nans)
mask = ~np.isnan(rh_array) & ~np.isnan(wl_array)    
slope, intercept, r_val, p_val, std_err = stats.linregress(rh_array[mask], wl_array[mask])
checkfit = slope*rh_array[mask] + intercept    
resids = [i - j for i, j in zip(wl_array[mask], checkfit)]
resids_array = np.array(resids)
rms_resids = getrms(resids_array)

#plot time series for the water levels and reflector heights (with reversed axes)    
fig, ax1 = plt.subplots(figsize=(10, 8))
color = 'tab:blue'
ax1.set_xlabel('Date', fontsize=16)
ax1.set_ylabel('Tide Gauge Water Level (m)', color='black', fontsize=16)
ax1.scatter(tidedates, waterlevel, label='Tide Gauge', color=color)
ax1.tick_params(axis='y', labelcolor='black', labelsize=14)
ax1.tick_params(axis='x', labelcolor='black', labelsize=14)
plt.ylim(-.5,.3)    
#instantiate a second axes that shares the same x-axis    
ax2 = ax1.twinx()  
color = 'tab:orange'
ax2.set_ylabel('GPS Reflector Height (m)', color='black', fontsize=16)  
#we already handled the x-label with ax1
ax2.scatter(mchndates, reflht, label='GPS Reflector Height', color=color)
ax2.tick_params(axis='y', labelcolor='black', labelsize=14)
plt.ylim(6.75,7.55)
plt.gca().invert_yaxis()
fig.legend(loc='lower right', bbox_to_anchor=(0.925, 0.079), edgecolor='black')
plt.title('MCHN Tide Gauge Measurements vs. Reflectometry', fontsize=18)    


#plot reflector height vs. water level (using masked values)    
fig, ax = plt.subplots(figsize=(10,8))
ax.plot(rh_array[mask], checkfit, '-', color='black')
ax.scatter(rh_array[mask], wl_array[mask], color = 'tab:blue')    
ax.set_xlabel("Reflector Height (m)", fontsize=16)
ax.set_ylabel("Water Level (m)", fontsize=16)
ax.set_title('MCHN Reflector Height vs. Tide Gauge Measurements', fontsize=18)
ax.tick_params(labelsize=14)
plt.grid()
txtstr = '\n'.join((
    'Slope=%.2f' % (slope, ),
    'Intercept=%.2f m' % (intercept, ),
    'Correlation=%.3f' % (r_val, ),
    'P-value=%.2f' % (p_val, ),
    'RMS of Residuals=%.3f m' % (rms_resids, )))
props = dict(boxstyle='round', facecolor='white', alpha=1)
ax.text(.65, .95, txtstr, transform=ax.transAxes, fontsize=14, verticalalignment='top', bbox=props)
fig.tight_layout()
plt.show()

The linear regression between the two series gives a slope m=-1.02. The rms of the residuals is very good, 0.026 m.

### Reference

DFO (2021). Institute of Ocean Sciences Data Archive. Ocean Sciences Division. Department of Fisheries and Oceans 
Canada. http://www.pac.dfo-mpo.gc.ca/science/oceans/data-donnees/index-eng.html. Data obtained on 2021-01-28.